## Scenario6 - drone count vs uncertainty-constant delta 0.5 - To dcoument the uncertainity in a table

In [11]:
from scipy.optimize import least_squares
import numpy as np
import time
from pandas import DataFrame
import math
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from scipy.stats import sem

### Method to generate drone position required for simulation

In [12]:
def fun_generate_drone_pos(drone_count):
    lpos = []
    for i in range(drone_count):
        xy=np.random.uniform(-10,10,2)
        x=xy[0]
        y=xy[1]
        z=np.random.uniform(2,7,1)[0]
        lpos.append(x)
        lpos.append(y)
        lpos.append(z)
    #print(lpos)
    return (np.asarray(lpos).reshape(1,drone_count, 3))
#fun_generate_drone_pos()

### Calculate the time-of-flight data with the actual drone position and ground object location

In [13]:
def calc_timelags_noisedp(n, dp_orig, dp_noise):
    json_loc = {}
    list_orig_y = []
    json1 = {}
    for i in range(n):
        for k in range(i, n):
            outer = i + 1
            inner = k + 1
            key = "" + str(outer) +"_"+ str(inner)

            if(i==k):
                val=  math.sqrt(sum((x_original-dp_orig[0][i])**2)) * 2 
                pos1 = dp_noise[0][i]
                pos2 = ""
                #print(key, " ", val)
            else: 
                #print(key , ",", "i = ", i, "k=", k)
                val = math.sqrt(sum((x_original-dp_orig[0][outer-1])**2)) + math.sqrt(sum((x_original-dp_orig[0][inner-1])**2))
                pos1 = dp_noise[0][outer-1]
                pos2 = dp_noise[0][inner-1]
                #print(key, " ", val)
            
            json11 = {key: val, "pos1":pos1, "pos2":pos2 }
            json1.update(json11)
            list_orig_y.append(json11)
            #json.append(json1)

    #print(json1)
    return list_orig_y
    
#y = [9.095598162479572,9.282884570730419,12.974780298767472,5.403702434442518,12.787493890516625,13.162066707018319]

### Calculate the difference bw actual and recovered object location

In [14]:
def calc_recored_go_pos_err(actual_go,recovered_go):
    sq_diff=((actual_go-recovered_go)**2)
    total_sum_go_err = np.sqrt(  sq_diff[0]) + np.sqrt( sq_diff[1]) + np.sqrt( sq_diff[2])
    return total_sum_go_err

### Objective function to recover the object location - the function takes input as initial guess of object location, drone coordinates, time-of-flight and LS method iteratively calls this function to minimize

In [15]:
def recover_GO_pos(x ):

    x[2]=0
    #print("hi--")
    list_orig_y = list_temp_json[0]
    #print((list_orig_y))
    listval =[]

    for json_y in list_orig_y:
        c = 0
        for key in json_y:
            # execute the logic only once per json obj(there are 3 keys), c=0 controls this logic
            if(c==0):
                #print("json--", json_y)
                #print("key--",key)
                v = json_y[key]
                #print("v===",v)
                k = key.split("_")
                k1 = k[0]
                k2 = k[1]
                
                k1=k1.strip()
                k1=int(k1)
                
                k2=k2.strip()
                k2=int(k2)

                if(k1==k2):
                    val=  v - math.sqrt(sum((x-json_y["pos1"])**2))* 2
                    listval.append(val)
                else:
                    val = v - (math.sqrt(sum((x-json_y["pos1"])**2)) + math.sqrt(sum((x-json_y["pos2"])**2)))
                    listval.append(val)
                    
            c = 1
    #print("listval---",listval)
    an_array = np.array(listval)
    an_array1 = an_array**2
    loss = sum(an_array1)
    #print(loss)
    #print("loss---",loss)
    return np.array(loss)


### Function to add noise to drone position

In [16]:
def addnoise_to_drone_pos(dp_orig, sd):
    #d_pos=np.array([[[4.3,4.3,0.8],[2.4,-1.2,5.4],[6.9,0.1,4.7]]])
    noise = np.random.normal(0, sd, dp_orig.shape)
    #print(noise)
    dp_noise = dp_orig + noise
    #d_pos=np.array([[[4.4,4.4,0.9],[2.4,-1.2,5.4],[6.9,0.1,4.7]]])
    return dp_noise
    #print(d_pos)  

### Variable initilization

In [17]:
x_original=np.array([2.8,2.2,0])
xy_initial = np.array([0,0,0]) 
arr_noise_sd = [0.5]

# Invoke LS with different drone count for noise level 0.5

# Drone Count = 3

In [18]:
drone_count=3
start_time = time.time()

list_temp_json = []

df_results3 = DataFrame(columns=['uncer_dp', 'uncer_rec_go', 'sd_rec_go_pos', 'std_err_rec_go_pos'])
ite = 0
for noise_sd in arr_noise_sd:
    list_goerror = []
    ite +=1
    for iterations in range(300):
        list_temp_json.clear()
        dp_real = fun_generate_drone_pos(drone_count)
        dp_noise = addnoise_to_drone_pos(dp_real,sd=noise_sd)
        list_orig_y = calc_timelags_noisedp(drone_count,dp_real, dp_noise)
        list_temp_json.insert(0, list_orig_y)

        result = least_squares(recover_GO_pos, xy_initial)
        err_go = calc_recored_go_pos_err(x_original,result.x)

        list_goerror.append(err_go)
        
    avg_err_go = np.mean(list_goerror)
    sd_rec_calc_gopos = np.std(list_goerror) # sd of rec go obj pos
    std_err_calc_gopos = sem(list_goerror)   # std dev of rec go obj pos = sd/sqrt(n-1)
    df_results3.loc[ite-1] = [noise_sd, avg_err_go, sd_rec_calc_gopos,std_err_calc_gopos]

print("--- %s seconds ---" % (time.time() - start_time))      


--- 7.10003662109375 seconds ---


In [21]:
len(list_temp_json)

1

# Drone Count = 6

In [19]:
drone_count=6
start_time = time.time()

list_temp_json = []

df_results6 = DataFrame(columns=['uncer_dp', 'uncer_rec_go', 'sd_rec_go_pos', 'std_err_rec_go_pos'])
ite = 0
for noise_sd in arr_noise_sd:
    list_goerror = []
    ite +=1
    for iterations in range(300):
        list_temp_json.clear()
        dp_real = fun_generate_drone_pos(drone_count)
        dp_noise = addnoise_to_drone_pos(dp_real,sd=noise_sd)
        list_orig_y = calc_timelags_noisedp(drone_count,dp_real, dp_noise)
        list_temp_json.insert(0, list_orig_y)

        result = least_squares(recover_GO_pos, xy_initial)
        err_go = calc_recored_go_pos_err(x_original,result.x)

        list_goerror.append(err_go)
        
    avg_err_go = np.mean(list_goerror)
    sd_rec_calc_gopos = np.std(list_goerror) # sd of rec go obj pos
    std_err_calc_gopos = sem(list_goerror)   # std dev of rec go obj pos = sd/sqrt(n-1)
    df_results6.loc[ite-1] = [noise_sd, avg_err_go, sd_rec_calc_gopos,std_err_calc_gopos]

print("--- %s seconds ---" % (time.time() - start_time))      


--- 8.385699272155762 seconds ---


# Drone Count = 9

In [20]:
drone_count=9
start_time = time.time()

list_temp_json = []

df_results9 = DataFrame(columns=['uncer_dp', 'uncer_rec_go', 'sd_rec_go_pos', 'std_err_rec_go_pos'])
ite = 0
for noise_sd in arr_noise_sd:
    list_goerror = []
    ite +=1
    for iterations in range(300):
        list_temp_json.clear()
        dp_real = fun_generate_drone_pos(drone_count)
        dp_noise = addnoise_to_drone_pos(dp_real,sd=noise_sd)
        list_orig_y = calc_timelags_noisedp(drone_count,dp_real, dp_noise)
        list_temp_json.insert(0, list_orig_y)

        result = least_squares(recover_GO_pos, xy_initial)
        err_go = calc_recored_go_pos_err(x_original,result.x)

        list_goerror.append(err_go)
        
    avg_err_go = np.mean(list_goerror)
    sd_rec_calc_gopos = np.std(list_goerror) # sd of rec go obj pos
    std_err_calc_gopos = sem(list_goerror)   # std dev of rec go obj pos = sd/sqrt(n-1)
    df_results9.loc[ite-1] = [noise_sd, avg_err_go, sd_rec_calc_gopos,std_err_calc_gopos]

print("--- %s seconds ---" % (time.time() - start_time))      


--- 12.603575229644775 seconds ---


# Drone Count = 12

In [21]:
drone_count=12
start_time = time.time()

list_temp_json = []

df_results12 = DataFrame(columns=['uncer_dp', 'uncer_rec_go', 'sd_rec_go_pos', 'std_err_rec_go_pos'])
ite = 0
for noise_sd in arr_noise_sd:
    list_goerror = []
    ite +=1
    for iterations in range(300):
        list_temp_json.clear()
        dp_real = fun_generate_drone_pos(drone_count)
        dp_noise = addnoise_to_drone_pos(dp_real,sd=noise_sd)
        list_orig_y = calc_timelags_noisedp(drone_count,dp_real, dp_noise)
        list_temp_json.insert(0, list_orig_y)

        result = least_squares(recover_GO_pos, xy_initial)
        err_go = calc_recored_go_pos_err(x_original,result.x)

        list_goerror.append(err_go)
        
    avg_err_go = np.mean(list_goerror)
    sd_rec_calc_gopos = np.std(list_goerror) # sd of rec go obj pos
    std_err_calc_gopos = sem(list_goerror)   # std dev of rec go obj pos = sd/sqrt(n-1)
    df_results12.loc[ite-1] = [noise_sd, avg_err_go, sd_rec_calc_gopos,std_err_calc_gopos]

print("--- %s seconds ---" % (time.time() - start_time))      


--- 18.61638855934143 seconds ---


# Drone Count = 30

In [ ]:

start_time = time.time()

list_temp_json = []

df_results30 = DataFrame(columns=['uncer_dp', 'uncer_rec_go', 'sd_rec_go_pos', 'std_err_rec_go_pos'])
ite = 0
for noise_sd in arr_noise_sd:
    list_goerror = []
    ite +=1
    for iterations in range(300):
        list_temp_json.clear()
        dp_real = fun_generate_drone_pos(drone_count)
        dp_noise = addnoise_to_drone_pos(dp_real,sd=noise_sd)
        list_orig_y = calc_timelags_noisedp(drone_count,dp_real, dp_noise)
        list_temp_json.insert(0, list_orig_y)

        result = least_squares(recover_GO_pos, xy_initial)
        err_go = calc_recored_go_pos_err(x_original,result.x)

        list_goerror.append(err_go)
        
    avg_err_go = np.mean(list_goerror)
    sd_rec_calc_gopos = np.std(list_goerror) # sd of rec go obj pos
    std_err_calc_gopos = sem(list_goerror)   # std dev of rec go obj pos = sd/sqrt(n-1)
    df_results30.loc[ite-1] = [noise_sd, avg_err_go, sd_rec_calc_gopos,std_err_calc_gopos]

print("--- %s seconds ---" % (time.time() - start_time))      


# Print uncertanty in recovered object for each drone count

In [23]:
df_results3

,uncer_dp,uncer_rec_go,sd_rec_go_pos,std_err_rec_go_pos
0,0.5,1.303087,1.258398,0.072775


In [27]:
df_results6

,uncer_dp,uncer_rec_go,sd_rec_go_pos,std_err_rec_go_pos
0,0.5,0.637295,0.679024,0.039269


In [26]:
df_results9

,uncer_dp,uncer_rec_go,sd_rec_go_pos,std_err_rec_go_pos
0,0.5,0.497038,0.30176,0.017451


In [25]:
df_results12

,uncer_dp,uncer_rec_go,sd_rec_go_pos,std_err_rec_go_pos
0,0.5,0.41321,0.227536,0.013159


In [24]:
df_results30

,uncer_dp,uncer_rec_go,sd_rec_go_pos,std_err_rec_go_pos
0,0.5,0.260564,0.142134,0.00822
